# DATA PREPARATION

The objective of our model is basically to predict the is_canceled column: https://www.kaggle.com/datasets/thedevastator/hotel-bookings-analysis.

Here we just apply the changes decided in the exploration part.

# Data preparation required after exploration:
**days_in_waiting_list** : Create 6 categories.
* 0
* less than a week
* less than two weeks
* less than a momths
* less than 3 months
* more.

**stays_in_weekend_nights**: Create a 'more than 5 stays_in_weekend_nights' category by replacing every number above 6 by 6.

**stays_in_week_nights**: Create a 'more than 5 stays_in_week_nights' by replacing every number above 6 by 6.

**previous_cancellations**: Create 4 groups.
* 0 day
* 1 day
*  2 - 6 days
*  7 + days

**previous_bookings_not_cancelled**:
* 0 previous bookings
* 1 - 5 previous bookings
* 6 + previous bookings

**booking_changes**:
* 0 booking changes
* 1 booking change
* 2 - 5 booking changes
* 6 + booking changes

**agent** : Keep IDs that represent more than 70% of the total reservations, and the rest will be considered as others.

**arrival_date**: Sort date by seasons (winter, spring, summer, autumn) and delete the arrival_date_month and arrival_date_day.

**meal**: Attribute 0 for SC (no meal), 1 for BB (one meal), 2 for HB and FB (two or more meals).

**country**: Keep only two categories : locals / non-locals.

**market_segement**: divided into four columns.
* Online TA
* Offline TA/TO
* Groups
* Others

**reserved_room_type**: Keep only three columns.
* A
* D
* Other


# Fetching data

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
drive.mount('/content/drive')

df = pd.read_pickle('/content/drive/My Drive/IAAE/data/explored_data.pkl')

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
Index: 78385 entries, 40060 to 119389
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   is_canceled                     78385 non-null  int64  
 1   lead_time                       78385 non-null  int64  
 2   arrival_date_year               78385 non-null  int64  
 3   arrival_date_week_number        78385 non-null  int64  
 4   stays_in_weekend_nights         78385 non-null  int64  
 5   stays_in_week_nights            78385 non-null  int64  
 6   adults                          78385 non-null  int64  
 7   children                        78385 non-null  float64
 8   meal                            78385 non-null  object 
 9   country                         78385 non-null  object 
 10  market_segment                  78385 non-null  object 
 11  is_repeated_guest               78385 non-null  int64  
 12  previo

In [ ]:
df = pd.read_pickle('/content/drive/My Drive/IAAE/data/explored_data.pkl')

# days_in_waiting_list

We create 6 categories : 0, less than a week, less than two weeks, less than a momths, less than 3 months, more than that.

In [ ]:
df.loc[(df['days_in_waiting_list'] == 0) , 'days_in_waiting_list'] = 0
df.loc[(df['days_in_waiting_list'] > 0) & (df['days_in_waiting_list'] <= 7) , 'days_in_waiting_list'] = 1
df.loc[(df['days_in_waiting_list'] > 7) & (df['days_in_waiting_list'] <= 14) , 'days_in_waiting_list'] = 2
df.loc[(df['days_in_waiting_list'] > 14) & (df['days_in_waiting_list'] <= 30) , 'days_in_waiting_list'] = 3
df.loc[(df['days_in_waiting_list'] > 30) & (df['days_in_waiting_list'] <= 90) , 'days_in_waiting_list'] = 4
df.loc[(df['days_in_waiting_list'] > 90) , 'days_in_waiting_list'] = 5

df["days_in_waiting_list"].value_counts()


,count
days_in_waiting_list,
0,74942
4,2042
5,843
3,384
1,112
2,62


# stays_in_weekend_nights

We replace every number of df["stays_in_weekend_nights"] above 6 by 6.

In [ ]:
df.loc[df['stays_in_weekend_nights'] > 6 , 'stays_in_weekend_nights'] = 6

df["stays_in_weekend_nights"].value_counts()

,count
stays_in_weekend_nights,
0,37424
1,21222
2,19067
3,298
4,267
6,68
5,39


# stays_in_week_nights

We replace every number of df["stays_in_weekend_nights"] above 6 by 6.

In [ ]:
df.loc[df['stays_in_week_nights'] > 6 , 'stays_in_week_nights'] = 6

df["stays_in_week_nights"].value_counts()

,count
stays_in_week_nights,
2,26196
1,20773
3,16254
4,6073
0,4875
5,3207
6,1007


# previous_cancellations

We create 4 groups : 0, 1, [2;6], >6

In [ ]:
df.loc[(df['previous_cancellations'] >= 2) & (df['previous_cancellations'] <= 6), 'previous_cancellations'] = 2
df.loc[(df['previous_cancellations'] > 6) , 'previous_cancellations'] = 3

df["previous_cancellations"].value_counts()

,count
previous_cancellations,
0,73000
1,5152
2,185
3,48


# previous_bookings_not_cancelled

In [ ]:
df.loc[(df['previous_bookings_not_canceled'] >= 1) & (df['previous_bookings_not_canceled'] <= 5), 'previous_bookings_not_canceled'] = 1
df.loc[(df['previous_bookings_not_canceled'] > 5) , 'previous_bookings_not_canceled'] = 2
df["previous_bookings_not_canceled"].value_counts()

,count
previous_bookings_not_canceled,
0,76820
1,1064
2,501


# booking_changes

In [ ]:
df.loc[(df['booking_changes'] >= 2) & (df['booking_changes'] <= 5), 'booking_changes'] = 2
df.loc[(df['booking_changes'] > 5) , 'booking_changes'] = 3

df["booking_changes"].value_counts()

,count
booking_changes,
0,68243
1,7156
2,2902
3,84


# agent

We keep the smallest quantity of IDs that represent 70% of the total rows. We merge all other IDs as a single category.

In [ ]:
id_counts = df['agent'].value_counts()

cumulative_sum = id_counts.cumsum()

#We select the IDs that cumulatively cover 70% of the data
total_counts = id_counts.sum()
threshold = total_counts * 0.7
selected_ids = id_counts[cumulative_sum <= threshold].index


#In df["agent"] we set all of the values that are not in selected_ids to 0
df.loc[~df['agent'].isin(selected_ids), 'agent'] = 0

df["agent"].value_counts()


,count
agent,
9.0,31581
0.0,29923
1.0,7137
14.0,3552
7.0,3511
6.0,2681


These IDs are categorical values hence we create separated columns.

In [ ]:
df = pd.get_dummies(df, columns=['agent'], prefix='agent')
df.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,agent_0.0,agent_1.0,agent_6.0,agent_7.0,agent_9.0,agent_14.0
40060,0,6,2015,27,0,2,1,0.0,HB,PRT,...,Transient,0.0,0,0,False,False,True,False,False,False
40061,1,88,2015,27,0,4,2,0.0,BB,PRT,...,Transient,76.5,0,1,False,False,False,False,True,False
40062,1,65,2015,27,0,4,1,0.0,BB,PRT,...,Transient,68.0,0,1,False,False,False,False,True,False
40063,1,92,2015,27,2,4,2,0.0,BB,PRT,...,Transient,76.5,0,2,False,False,False,False,True,False
40064,1,100,2015,27,0,2,2,0.0,BB,PRT,...,Transient,76.5,0,1,False,False,False,False,True,False


# arrival_date_week_number

We create 4 categories (0, 1, 2, 3) that correspond to the four seasons.

In [ ]:
df.loc[(df['arrival_date_week_number'] <= 13) , 'arrival_date_week_number'] = 0
df.loc[(df['arrival_date_week_number'] >= 14) & (df['arrival_date_week_number'] <= 26) , 'arrival_date_week_number'] = 1
df.loc[(df['arrival_date_week_number'] >= 27) & (df['arrival_date_week_number'] <= 39) , 'arrival_date_week_number'] = 2
df.loc[(df['arrival_date_week_number'] >= 40), 'arrival_date_week_number'] = 3

df["arrival_date_week_number"].value_counts()


,count
arrival_date_week_number,
2,23585
1,23222
3,17121
0,14457


From this we create separated columns (one for each season)

In [ ]:
# Rename the "arrival_date_week_number" column to "season"
df.rename(columns={'arrival_date_week_number': 'season'}, inplace=True)
df.head()

# Get dummies for df["season"]
df = pd.get_dummies(df, columns=['season'], prefix='season')
df.head()

,is_canceled,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,market_segment,...,agent_0.0,agent_1.0,agent_6.0,agent_7.0,agent_9.0,agent_14.0,season_0,season_1,season_2,season_3
40060,0,6,2015,0,2,1,0.0,HB,PRT,Offline TA/TO,...,False,False,True,False,False,False,False,False,True,False
40061,1,88,2015,0,4,2,0.0,BB,PRT,Online TA,...,False,False,False,False,True,False,False,False,True,False
40062,1,65,2015,0,4,1,0.0,BB,PRT,Online TA,...,False,False,False,False,True,False,False,False,True,False
40063,1,92,2015,2,4,2,0.0,BB,PRT,Online TA,...,False,False,False,False,True,False,False,False,True,False
40064,1,100,2015,0,2,2,0.0,BB,PRT,Online TA,...,False,False,False,False,True,False,False,False,True,False


# meal

We attribute numerical values to the catgories according to the number of meals.

0.   SC
1.   BB
2.   HB or FB





In [ ]:

df.loc[df['meal'] == 'SC' , 'meal'] = 0
df.loc[df['meal'] == 'BB' , 'meal'] = 1
df.loc[(df['meal'] == 'HB') | (df['meal'] == 'FB') , 'meal'] = 2

df["meal"].value_counts()

,count
meal,
1,61554
0,10392
2,6439


# country

We keep only two categories : locals (Portugal) / non-locals.

The column is renamed "is_local" for more clarity and contains only booleans.

In [ ]:
df["country"] = df["country"].apply(lambda x: 1 if x == 'PRT' else 0)
df = df.rename(columns={'country': 'is_local'})
df["is_local"].value_counts()

,count
is_local,
0,47957
1,30428


# market_segment

We divide df["market_segment"] into four categories :  "Online TA", "Offline TA/TO", "Groups", Others

In [ ]:
df.loc[~df['market_segment'].isin(['Online TA', 'Offline TA/TO', 'Groups']) , 'market_segment'] = 3
df.loc[df['market_segment'] == 'Online TA' , 'market_segment'] = 0
df.loc[df['market_segment'] == 'Offline TA/TO' , 'market_segment'] = 1
df.loc[df['market_segment'] == 'Groups' , 'market_segment'] = 2


df["market_segment"].value_counts()

,count
market_segment,
0,38290
1,16548
2,13915
3,9632


Those are categorical values so we separate the columns

In [ ]:
df = pd.get_dummies(df, columns=['market_segment'], prefix='market_segment')
df.head()

,is_canceled,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,is_local,is_repeated_guest,...,agent_9.0,agent_14.0,season_0,season_1,season_2,season_3,market_segment_0,market_segment_1,market_segment_2,market_segment_3
40060,0,6,2015,0,2,1,0.0,2,1,0,...,False,False,False,False,True,False,False,True,False,False
40061,1,88,2015,0,4,2,0.0,1,1,0,...,True,False,False,False,True,False,True,False,False,False
40062,1,65,2015,0,4,1,0.0,1,1,0,...,True,False,False,False,True,False,True,False,False,False
40063,1,92,2015,2,4,2,0.0,1,1,0,...,True,False,False,False,True,False,True,False,False,False
40064,1,100,2015,0,2,2,0.0,1,1,0,...,True,False,False,False,True,False,True,False,False,False


# reserved_room_type

We divide df["reserved_room_type"] into three categories : "A", "D", Other

In [ ]:
df.loc[~df['reserved_room_type'].isin(['A', 'D']) , 'reserved_room_type'] = 2
df.loc[df['reserved_room_type'] == 'A' , 'reserved_room_type'] = 0
df.loc[df['reserved_room_type'] == 'D' , 'reserved_room_type'] = 1

df["reserved_room_type"].value_counts()

,count
reserved_room_type,
0,61882
1,11625
2,4878


Those are categorical values so we separate the columns

In [ ]:
df = pd.get_dummies(df, columns=['reserved_room_type'], prefix='reserved_room_type')
df.head()

,is_canceled,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,is_local,is_repeated_guest,...,season_1,season_2,season_3,market_segment_0,market_segment_1,market_segment_2,market_segment_3,reserved_room_type_0,reserved_room_type_1,reserved_room_type_2
40060,0,6,2015,0,2,1,0.0,2,1,0,...,False,True,False,False,True,False,False,True,False,False
40061,1,88,2015,0,4,2,0.0,1,1,0,...,False,True,False,True,False,False,False,True,False,False
40062,1,65,2015,0,4,1,0.0,1,1,0,...,False,True,False,True,False,False,False,True,False,False
40063,1,92,2015,2,4,2,0.0,1,1,0,...,False,True,False,True,False,False,False,True,False,False
40064,1,100,2015,0,2,2,0.0,1,1,0,...,False,True,False,True,False,False,False,True,False,False


# deposit_type

We keep every category.

Categorical values so we separate the columns.

In [ ]:
df = pd.get_dummies(df, columns=['deposit_type'], prefix='deposit_type')
df.head()

,is_canceled,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,is_local,is_repeated_guest,...,market_segment_0,market_segment_1,market_segment_2,market_segment_3,reserved_room_type_0,reserved_room_type_1,reserved_room_type_2,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable
40060,0,6,2015,0,2,1,0.0,2,1,0,...,False,True,False,False,True,False,False,True,False,False
40061,1,88,2015,0,4,2,0.0,1,1,0,...,True,False,False,False,True,False,False,True,False,False
40062,1,65,2015,0,4,1,0.0,1,1,0,...,True,False,False,False,True,False,False,True,False,False
40063,1,92,2015,2,4,2,0.0,1,1,0,...,True,False,False,False,True,False,False,True,False,False
40064,1,100,2015,0,2,2,0.0,1,1,0,...,True,False,False,False,True,False,False,True,False,False


# customer_type

Categorical values so we separate the columns.

In [ ]:
df = pd.get_dummies(df, columns=['customer_type'], prefix='customer_type')
df.head()

,is_canceled,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,is_local,is_repeated_guest,...,reserved_room_type_0,reserved_room_type_1,reserved_room_type_2,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
40060,0,6,2015,0,2,1,0.0,2,1,0,...,True,False,False,True,False,False,False,False,True,False
40061,1,88,2015,0,4,2,0.0,1,1,0,...,True,False,False,True,False,False,False,False,True,False
40062,1,65,2015,0,4,1,0.0,1,1,0,...,True,False,False,True,False,False,False,False,True,False
40063,1,92,2015,2,4,2,0.0,1,1,0,...,True,False,False,True,False,False,False,False,True,False
40064,1,100,2015,0,2,2,0.0,1,1,0,...,True,False,False,True,False,False,False,False,True,False


# Scaling

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78385 entries, 40060 to 119389
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   is_canceled                     78385 non-null  int64  
 1   lead_time                       78385 non-null  int64  
 2   arrival_date_year               78385 non-null  int64  
 3   stays_in_weekend_nights         78385 non-null  int64  
 4   stays_in_week_nights            78385 non-null  int64  
 5   adults                          78385 non-null  int64  
 6   children                        78385 non-null  float64
 7   meal                            78385 non-null  object 
 8   is_local                        78385 non-null  int64  
 9   is_repeated_guest               78385 non-null  int64  
 10  previous_cancellations          78385 non-null  int64  
 11  previous_bookings_not_canceled  78385 non-null  int64  
 12  booking_changes                 

In [ ]:
# Scale df using mim-max scaling from 0 to 1
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled.head()

,is_canceled,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,is_local,is_repeated_guest,...,reserved_room_type_0,reserved_room_type_1,reserved_room_type_2,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0.0,0.009539,0.0,0.000000,0.333333,0.25,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.139905,0.0,0.000000,0.666667,0.50,0.0,0.5,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.103339,0.0,0.000000,0.666667,0.25,0.0,0.5,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.146264,0.0,0.333333,0.666667,0.50,0.0,0.5,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.158983,0.0,0.000000,0.333333,0.50,0.0,0.5,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# Checking that everything is prepared

We export df_scaled.describe() to csv to be able to visualize every column and check if we didnt forget anything.

In [ ]:
df_scaled.describe().to_csv('/content/drive/My Drive/IAAE/data/prepared_data_describe.csv')

# Separating test and training sets

In [ ]:
X = df_scaled.drop('is_canceled',axis=1)
y = df_scaled['is_canceled']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=13,stratify=y)

# Saving test and training sets

In [ ]:
X_train.to_pickle('/content/drive/My Drive/IAAE/data/train/X_train.pkl')
y_train.to_pickle('/content/drive/My Drive/IAAE/data/train/y_train.pkl')
X_test.to_pickle('/content/drive/My Drive/IAAE/data/test/X_test.pkl')
y_test.to_pickle('/content/drive/My Drive/IAAE/data/test/y_test.pkl')